In [1]:
# Install dependencies
!pip -q install -U datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 8.5 MB/s eta 0:00:00


In [2]:
# Login
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, DatasetDict

# Load all splits
ds_all = load_dataset("mohammad-shirkhani/social_movielens")

# quick summary 
ds_all

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/910 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/57.5M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/57.5M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/57.2M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/80000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['user', 'item', 'answer', 'paths'],
        num_rows: 80000
    })
    validation: Dataset({
        features: ['user', 'item', 'answer', 'paths'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['user', 'item', 'answer', 'paths'],
        num_rows: 10000
    })
})

In [4]:
# Determine split keys robustly
available_splits = list(ds_all.keys())
print("Available splits:", available_splits)

for split in available_splits:
    print(f"Columns in '{split}':", ds_all[split].column_names)
    print(f"Num rows in '{split}':", len(ds_all[split]))
    print("-"*60)

Available splits: ['train', 'validation', 'test']
Columns in 'train': ['user', 'item', 'answer', 'paths']
Num rows in 'train': 80000
------------------------------------------------------------
Columns in 'validation': ['user', 'item', 'answer', 'paths']
Num rows in 'validation': 10000
------------------------------------------------------------
Columns in 'test': ['user', 'item', 'answer', 'paths']
Num rows in 'test': 10000
------------------------------------------------------------


In [5]:
# Shuffle
SEED = 42

# Create a new DatasetDict with shuffled splits
ds_shuffled = DatasetDict({name: ds_all[name].shuffle(seed=SEED) for name in ds_all.keys()})

# Sanity checks: lengths and columns should match exactly
for name in ds_all.keys():
    assert len(ds_shuffled[name]) == len(ds_all[name]), f"Row count mismatch in split '{name}'"
    assert ds_shuffled[name].column_names == ds_all[name].column_names, f"Columns changed in split '{name}'"
print("Shuffling complete. Schema preserved and lengths unchanged.")

Shuffling complete. Schema preserved and lengths unchanged.


In [6]:
example_split = available_splits[0]
print("First 2 indices before shuffle:", list(range(min(2, len(ds_all[example_split])))))
print("First 2 rows after shuffle (indices are randomized order):")
ds_shuffled[example_split].select(range(min(2, len(ds_shuffled[example_split]))))

First 2 indices before shuffle: [0, 1]
First 2 rows after shuffle (indices are randomized order):


Dataset({
    features: ['user', 'item', 'answer', 'paths'],
    num_rows: 2
})

In [7]:
from huggingface_hub import HfApi, create_repo, whoami

api = HfApi()
me = api.whoami()
username = me['name']
repo_id = f"{username}/social_movielens_new"

print("Logged in as:", username)
print("Target dataset repo:", repo_id)

# Create the dataset repo 
create_repo(repo_id, repo_type="dataset", exist_ok=True, private=False)

# Push the shuffled DatasetDict
push_info = ds_shuffled.push_to_hub(repo_id)
push_info

Logged in as: mohammad-shirkhani
Target dataset repo: mohammad-shirkhani/social_movielens_new


Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mohammad-shirkhani/social_movielens_new/commit/476ea55f0c51c1914a28e12042e5ed17636b5ba7', commit_message='Upload dataset', commit_description='', oid='476ea55f0c51c1914a28e12042e5ed17636b5ba7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mohammad-shirkhani/social_movielens_new', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mohammad-shirkhani/social_movielens_new'), pr_revision=None, pr_num=None)